### Homework 1: COVID_19 Cases Prediction(Regression)


#### Step1: Download data（数据下载）

In [50]:
!gdown --id '1kLSW_-cW2Huj7bh84YTdimGBOJaODiOS' --output covid.train.csv
!gdown --id '1iiI5qROrAhZn-o4FPqsE97bMzDEFvIdg' --output covid.test.csv

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1kLSW_-cW2Huj7bh84YTdimGBOJaODiOS
To: /content/covid.train.csv
100% 2.49M/2.49M [00:00<00:00, 16.3MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1iiI5qROrAhZn-o4FPqsE97bMzDEFvIdg
To: /content/covid.test.csv
100% 993k/993k [00:00<00:00, 8.05MB/s]


#### Step2: Import Package（api包引入）

In [51]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar（进度条库）
from tqdm import tqdm

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

#### Step3: Some Utility Functions（公用方法）

In [52]:
def same_seed(seed):
  '''
  Fixes random number generator seeds for reproducibility
  修复随机生成器的种子以提高可重复性
  '''
  torch.backends.cudnn.deterministic = True # 为True，会使cuDNN只使用确定的卷积算法。 
  torch.backends.cudnn.benchmark = False # 为True,cuDNN将对多种卷积算法进行基准测试，并选择最快的算法
  np.random.seed(seed) # 使用np.random.seed(),规使每次得到的随机数一致
  torch.manual_seed(seed) # 使用torch.manual_seed()，使每次得到的随机数一致
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed) # 在GPU环境下为所有的GPU设置种子，但不能使每次得到的随机数一致，如果要一直，需要每次产生随机数的时候都调用torch.cuda.manual_seed()

def train_valid_split(data_set,valid_ratio,seed):
  '''
  Split provided training data into training set and validation set
  将提供的训练数据分成训练集和验证集
  '''
  valid_set_size = int(valid_ratio * len(data_set)) 
  train_set_size = len(data_set) - valid_set_size
  train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
  return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
  model.eval() # 将自己的模型设置为评估模型
  preds = [] # 预测结果
  for x in tqdm(test_loader):
    x = x.to(device) # 把test_loader中的数据，上载到设备上
    with torch.no_grad():# 测试集，不计算梯度
      pred = model(x)
      preds.append(pred.detach().cpu()) #.detach(): 返回一个新的tensor，从当前图形中分离出来，该结果将永远不需要梯度。
  preds = torch.cat(preds,dim=0).numpy() # numpy不能读取CUDA tensor，需要先转换为CPU tensor
  return preds


###### PS：
torch.utils.data.random_split用法：
torch.utils.data.random_split(dataset, lengths, generator=<torch._C.Generator object>)
- dataset (Dataset) – 要划分的数据集。
- lengths (sequence) – 要划分的长度。
- generator (Generator) – 用于随机排列的生成器。

#### Step4: 定义Dataset

In [53]:
class COVID19Dataset(Dataset):
  '''
  x: Features.
  y: Targets, if none, do prediction.
  '''
  def __init__(self, x, y=None):
    if y is None:
      self.y = y
    else:
      self.y = torch.FloatTensor(y)
    self.x = torch.FloatTensor(x)
  def __getitem__(self, idx):
    if self.y is None:
      return self.x[idx]
    else:
      return self.x[idx], self.y[idx]
  
  def __len__(self):
    return len(self.x)

#### Step5: Nerual Network Model


In [54]:
class My_Model(nn.Module):
  def __init__(self, input_dim):
    super(My_Model, self).__init__()
    self.layers = nn.Sequential(
        # 神经网络结构搭建
        nn.Linear(input_dim, 32),
        nn.ReLU(),
        nn.Linear(32, 16), 
        nn.ReLU(),
        nn.Linear(16, 8),
        nn.ReLU(),
        nn.Linear(8, 1)
        # 隐藏层1 dim =16， #隐藏层2 dim = 8
    )
  
  def forward(self, x):
    # 前向传播方法定义
    x = self.layers(x)
    x = x.squeeze(1) #输出的array,shape是[n,1],squeeze成[n,]
    return x

#### Step6: Feature Selection（特征选取）

In [55]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1] # 最后一行为y,y_train,y_valid shape为(n,)
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data #raw_x,shape为(n,d),d为x参数个数

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = list(range(38,raw_x_train.shape[1])) # 通过改变feat_idx,选取合适的x参数进行训练与测试
        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

#### Step7: Training Loop（training 迭代）

In [56]:
def trainer(train_loader, valid_loader, model, config, device):

  # loss function（损失函数）选取
  criterion = nn.MSELoss(reduction='mean')
  # optimization algorithm（优化算法）选取
  optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.99)
  # Writer of tensoboard
  writer = SummaryWriter()
  # Create directory of saving models（模型地址）
  if not os.path.isdir('./models'):
    os.mkdir('./models')
  # epochs数目，最优loss,step以及early_stop_count的定义
  n_epochs, best_loss, step, early_stop_count = config['n_epochs'],math.inf,0,0

  for epoch in range(n_epochs):
    '''set model to train mode'''
    model.train()
    loss_record = [] # loss记录数组
    train_pbar = tqdm(train_loader, position=0, leave=True) #tqdm，训练进度可视化

    for x, y in train_pbar:
      optimizer.zero_grad() #set gradient to zero
      x, y = x.to(device), y.to(device) # move data to device
      pred = model(x) # do prediction
      loss = criterion(pred, y) # compute loss
      loss.backward() # compute gradient(backpropagation)
      optimizer.step() #update parameters
      loss_record.append(loss.detach().item())

      # Display current epoch number and loss on tqdm progress bar.
      train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
      train_pbar.set_postfix({'loss': loss.detach().item()})

    mean_train_loss = sum(loss_record)/len(loss_record)
    writer.add_scalar('Loss/train', mean_train_loss, step)

    '''set model to evaluation mode'''
    model.eval()
    loss_record = []
    for x, y in valid_loader:
      x, y = x.to(device), y.to(device)
      with torch.no_grad(): #valid data不计算梯度
        pred = model(x)
        loss = criterion(pred, y)

      loss_record.append(loss.item())
    mean_valid_loss = sum(loss_record)/len(loss_record)
    print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss: .4f}')
    writer.add_scalar('Loss/valid', mean_valid_loss, step)

    if mean_valid_loss < best_loss:
      best_loss = mean_valid_loss
      torch.save(model.state_dict(), config['save_path']) # save best model
      print('Saving model with loss {:.3f}...'.format(best_loss))
      early_stop_count = 0
    else:
      early_stop_count += 1
    
    if early_stop_count > config['early_stop']:
      print('\nModel is not improving, so we halt the training session.')
      return

#### Step8: Configurations
`config`contains hyper-parameters for training and the path to save the model.

In [57]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 6000000,      # Your seed number, you can pick your lucky number. :)
    'select_all': False,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 2000,     # Number of epochs.            
    'batch_size': 512, 
    'learning_rate': 1e-5,              
    'early_stop': 400,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

#### Step9: Dataloader
Read data from files and set up training,validation,and testing sets.

In [58]:
# Set seed for reproducibility
same_seed(config['seed'])


# train_data size: 2699 x 118 (id + 37 states + 16 features x 5 days) 
# test_data size: 1078 x 117 (without last day's positive rate)
train_data, test_data = pd.read_csv('./covid.train.csv').values, pd.read_csv('./covid.test.csv').values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), COVID19Dataset(x_valid, y_valid), COVID19Dataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

train_data size: (2160, 118) 
valid_data size: (539, 118) 
test_data size: (1078, 117)
number of features: 79


#### Step10: Start training

In [59]:
model = My_Model(input_dim=x_train.shape[1]).to(device) # put the model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/2000]: 100%|██████████| 5/5 [00:00<00:00, 78.05it/s, loss=179]


Epoch [1/2000]: Train loss: 180.5146, Valid loss:  177.8506
Saving model with loss 177.851...


Epoch [2/2000]: 100%|██████████| 5/5 [00:00<00:00, 82.09it/s, loss=139]


Epoch [2/2000]: Train loss: 139.8124, Valid loss:  112.9580
Saving model with loss 112.958...


Epoch [3/2000]: 100%|██████████| 5/5 [00:00<00:00, 81.16it/s, loss=103]


...

Epoch [1741/2000]: Train loss: 1.1926, Valid loss:  0.9102


Epoch [1742/2000]: 100%|██████████| 5/5 [00:00<00:00, 76.85it/s, loss=1.33]


Epoch [1742/2000]: Train loss: 1.2117, Valid loss:  1.1712


Epoch [1743/2000]: 100%|██████████| 5/5 [00:00<00:00, 55.73it/s, loss=0.975]


Epoch [1743/2000]: Train loss: 1.1216, Valid loss:  1.2744

Model is not improving, so we halt the training session.


#### Step11: Plot learning curves with `tensorboard`


In [60]:
# %reload_ext tensorboard
# %tensorboard --logdir=./runs/

#### Step12: Testing

In [61]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device) 
save_pred(preds, 'pred.csv')         

100%|██████████| 3/3 [00:00<00:00, 256.11it/s]
